<a href="https://colab.research.google.com/github/kridtapon/TrixEFI-Flow/blob/main/TrixEFI_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.0 MB/s eta 0:00:00


In [13]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Trix Indicator (Triple Exponential Moving Average)
def calculate_trix(df, period=14):
    """
    Calculate the Trix Indicator.
    """
    # First, calculate the Exponential Moving Averages (EMA)
    ema1 = df['Close'].ewm(span=period).mean()
    ema2 = ema1.ewm(span=period).mean()
    ema3 = ema2.ewm(span=period).mean()

    # Trix is the rate of change of the third EMA
    trix = ema3.pct_change() * 100  # Multiply by 100 to express it as a percentage

    return trix

# Function to calculate Elder's Force Index (EFI)
def calculate_efi(df, period=13):
    """
    Calculate Elder's Force Index (EFI).
    """
    efi = (df['Close'] - df['Close'].shift(1)) * df['Volume']
    efi = efi.rolling(window=period).mean()  # Smoothing over the specified period
    return efi

# Define the stock symbol and time period
symbol = 'APO'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate Trix Indicator
df['Trix'] = calculate_trix(df, period=14)

# Calculate Elder's Force Index (EFI)
df['EFI'] = calculate_efi(df, period=13)

# Define Entry and Exit signals based on Trix and EFI
df['Entry'] = (df['Trix'] > 0) & (df['EFI'] > 0)  # Trix above 0 and EFI positive for entry
df['Exit'] = (df['Trix'] < 0) & (df['EFI'] < 0)   # Trix below 0 and EFI negative for exit

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           266552.755981
Total Return [%]                       166.552756
Benchmark Return [%]                   302.647021
Max Gross Exposure [%]                      100.0
Total Fees Paid                       4052.005066
Max Drawdown [%]                         31.47097
Max Drawdown Duration                       474.0
Total Trades                                   14
Total Closed Trades                            13
Total Open Trades                               1
Open Trade PnL                       77185.700984
Win Rate [%]                            61.538462
Best Trade [%]                          29.245869
Worst Trade [%]                        -16.662678
Avg Winning Trade [%]                   14.986988
Avg Losing Trade [%]                    -8.543291
